In [1]:
import numpy as np
import ipywidgets as widgets
import pylab as pl

In [2]:
def explicit_flow_solve(u, g, conditions):
    def __b(v, bounds):
        if v < bounds[0]:
            return 0
        if v > bounds[1]:
            return bounds[1]
        return v

    def __Lh(u, g, dx, Tj):
        return -u * ((Tj["i+1"] - Tj["i"]) / dx) + g * ((Tj["i-1"] + Tj["i+1"] - 2 * Tj["i"]) / dx / dx)

    x0, t0 = conditions["x_min"], conditions["t_min"]
    xn, tm = conditions["x_max"], conditions["t_max"]
    dt, dx = conditions['dt'], conditions['dx']
    T0 = conditions["T"]

    xs, ts = np.arange(x0, xn, dx), np.arange(t0, tm, dt)
    xl, tl = len(xs), len(ts)

    T = np.zeros([xl, tl])
    for i in range(xl):  # T[x][t]
        T[i][0] = T0(xs[i])

    for j in range(tl - 1):
        for i in range(xl):
            Tj = {
                "i-1": T[__b(i - 1, (0, xl - 1))][j],
                "i": T[i][j],
                "i+1": T[__b(i + 1, (0, xl - 1))][j],
            }

            T[i][j + 1] = __Lh(
                u(ts[j], xs[i], T[i][j]),
                g(ts[j], xs[i], T[i][j]),
                dx, Tj) * dt + T[i][j]

    return T


In [3]:
def explicit_counter_flow_solve(u, g, conditions):
    def __b(v, bounds):
        if v < bounds[0]:
            return 0
        if v > bounds[1]:
            return bounds[1]
        return v

    def __Lh(u, g, dx, Tj):
        return -u * ((Tj["i"] - Tj["i-1"]) / dx) + g * ((Tj["i-1"] + Tj["i+1"] - 2 * Tj["i"]) / dx / dx)
    
    x0, t0 = conditions["x_min"], conditions["t_min"]
    xn, tm = conditions["x_max"], conditions["t_max"]
    dt, dx = conditions['dt'], conditions['dx']
    T0 = conditions["T"]

    xs, ts = np.arange(x0, xn, dx), np.arange(t0, tm, dt)
    xl, tl = len(xs), len(ts)

    T = np.zeros([xl, tl])
    for i in range(xl):  # T[x][t]
        T[i][0] = T0(xs[i])

    for j in range(0, tl - 1):
        for i in range(xl):
            Tj = {
                "i-1": T[__b(i - 1, (0, xl - 1))][j],
                "i": T[i][j],
                "i+1": T[__b(i + 1, (0, xl - 1))][j],
            }

            T[i][j + 1] = __Lh(
                u(ts[j], xs[i], T[i][j]),
                g(ts[j], xs[i], T[i][j]),
                dx, Tj) * dt + T[i][j]

    return T

In [4]:
def leapfrog(u, _, conditions):
    def __b(v, bounds):
        if v < bounds[0]:
            return 0
        if v > bounds[1]:
            return bounds[1]
        return v

    def __Lh(u, g, dx, Tj):
        return -u * ((Tj["i+1"] - Tj["i-1"]) / (2 * dx)) + g * ((Tj["i-1"] + Tj["i+1"] - 2 * Tj["i"]) / dx / dx)
    
    x0, t0 = conditions["x_min"], conditions["t_min"]
    xn, tm = conditions["x_max"], conditions["t_max"]
    dt, dx = conditions['dt'], conditions['dx']
    T0 = conditions["T"]

    xs, ts = np.arange(x0, xn, dx), np.arange(t0, tm, dt)
    xl, tl = len(xs), len(ts)

    T = np.zeros([xl, tl])
    for i in range(xl):  # T[x][t]
        T[i][0] = T0(xs[i])

    for j in range(tl - 1):
        for i in range(xl):
            Tj = {
                "i-1": T[__b(i - 1, (0, xl - 1))][j],
                "i": T[i][j],
                "i+1": T[__b(i + 1, (0, xl - 1))][j],
            }

            T[i][j + 1] = __Lh(
                u(ts[j], xs[i], T[i][j]), 0,
                dx, Tj) * 2 * dt + T[i][__b(j - 1, (0, tl - 1))]

    return T

In [5]:
def implicit_counter_flow(u, g, conditions):
    x0, t0 = conditions["x_min"], conditions["t_min"]
    xn, tm = conditions["x_max"], conditions["t_max"]
    dt, dx = conditions['dt'], conditions['dx']
    T0 = conditions["T"]

    xs, ts = np.arange(x0, xn, dx), np.arange(t0, tm, dt)
    xl, tl = len(xs), len(ts)

    T = np.zeros([xl, tl])
    for i in range(xl):  # T[x][t]
        T[i][0] = T0(xs[i])

    for j in range(0, tl - 1):
        m = np.zeros((xl, xl))
        b = np.zeros(xl)
        for i in range(xl):
            s = u(ts[j], xs[i], T[i][j]) * dt / dx
            r = g(ts[j], xs[i], T[i][j]) * dt / dx / dx

            b[i] = T[i][j]
            m[i][max(i - 1, 0)] = -(s + r)
            m[i][min(i + 1, xl - 1)] = -r
            m[i][i] = (1 + s + 2 * r)

        tx = __solve_diagonal(m, b)
        for i in range(len(tx)):
            T[i][j + 1] = tx[i]

    return T


def __solve_diagonal(m, d):
    n = len(d) - 1
    xs = np.zeros(n + 1)

    als, bls = np.zeros(n + 1), np.zeros(n + 1)
    als[0] = - m[0][1] / m[0][0]
    bls[0] = - d[0] / m[0][0]

    for i in range(1, n):
        a = m[i][i - 1]
        b = m[i][i]
        c = m[i][i + 1]
        als[i] = - c / (a * als[i - 1] + b)
        bls[i] = (d[i] - a * bls[i - 1]) / (a * als[i - 1] + b)

    xs[n] = (d[n] - m[n][n - 1] * m[n - 1][n - 1]) / (m[n][n - 1] * als[n - 1] + m[n][n])

    for i in range(n - 1, -1, -1):
        xs[i] = als[i] * xs[i + 1] + bls[i]
    return xs

In [41]:
def get_flash_light():
    u = lambda t, x, T: 0.
    g = lambda t, x, T: 0.22
    conditions = {
        "t_min": 0.,
        "t_max": 50.,
        "dt": 0.5,
        "dx": 0.5,
        "T": lambda x: np.math.exp(-abs(x / 10)) * 10,
        "x_min": -100.,
        "x_max": 100.
    }
    return u, g, conditions


def get_steps():
    u = lambda t, x, T: 0
    g = lambda t, x, T: 0.22
    conditions = {
        "t_min": 0.,
        "t_max": 50.,
        "dt": 0.5,
        "dx": 0.5,
        "T": lambda x: x > 0,
        "x_min": -100.,
        "x_max": 100.
    }
    return u, g, conditions


def get_waves():
    u = lambda t, x, T: 1
    g = lambda t, x, T: 0.
    conditions = {
        "t_min": 0.,
        "t_max": 50.,
        "dt": 0.5,
        "dx": 0.5,
        "T": lambda x: (np.math.cos(x / 20) + 1) * 50,
        "x_min": -100.,
        "x_max": 100.
    }
    return u, g, conditions


def get_rhombus():
    u = lambda t, x, T: 1
    g = lambda t, x, T: 0.
    conditions = {
        "t_min": 0.,
        "t_max": 50.,
        "dt": 0.5,
        "dx": 0.5,
        "T": lambda x: x > 0 and x < 2.5,
        "x_min": -100.,
        "x_max": 100.
    }
    return u, g, conditions

def get_test():
    u = lambda t, x, T: 0.
    g = lambda t, x, T: 0.26
    conditions = {
        "t_min": 0.,
        "t_max": 50.,
        "dt": 0.5,
        "dx": 0.5,
        "T": lambda x: np.math.exp(-abs(x / 10)) * 10,
        "x_min": -100.,
        "x_max": 100.
    }
    return u, g, conditions

In [42]:
def show_mesh(method=implicit_counter_flow, initial_condition=get_flash_light):
    u, g, conditions = initial_condition()
    
    s = u(conditions["t_min"], conditions["x_min"], conditions["T"](conditions["x_min"])) * conditions["dt"] / conditions["dx"]
    r = g(conditions["t_min"], conditions["x_min"], conditions["T"](conditions["x_min"])) * conditions["dt"] / conditions["dx"] / conditions["dx"]
    print (s, r)
    
    cs = method(u, g, conditions)

    m_x, m_y = np.mgrid[
        slice(conditions["x_min"], conditions["x_max"], conditions["dx"]),
        slice(conditions["t_min"], conditions["t_max"], conditions["dt"])]

    pl.pcolormesh(m_x, m_y, cs)
    pl.show()

w = widgets.interactive(show_mesh,
                        method={'Explicit Flow': explicit_flow_solve, 
                                'Explicit Counter Flow': explicit_counter_flow_solve, 
                                'Leapfrog': leapfrog,
                                'Implicit Counter Flow': implicit_counter_flow},
                        initial_condition={'Flash light': get_flash_light, 
                                           'Steps': get_steps, 
                                           'Waves': get_waves, 
                                           'Rhombus': get_rhombus,
                                          'Test': get_test}
                       )
display(w)

interactive(children=(Dropdown(description='method', index=3, options={'Explicit Flow': <function explicit_flow_solve at 0x10a3438c8>, 'Explicit Counter Flow': <function explicit_counter_flow_solve at 0x112384400>, 'Leapfrog': <function leapfrog at 0x11243a048>, 'Implicit Counter Flow': <function implicit_counter_flow at 0x1122eff28>}, value=<function implicit_counter_flow at 0x1122eff28>), Dropdown(description='initial_condition', options={'Flash light': <function get_flash_light at 0x116573e18>, 'Steps': <function get_steps at 0x1160b07b8>, 'Waves': <function get_waves at 0x1160b0620>, 'Rhombus': <function get_rhombus at 0x1160b0158>, 'Test': <function get_test at 0x11243a6a8>}, value=<function get_flash_light at 0x116573e18>), Output()), _dom_classes=('widget-interact',))

In [40]:
def show_wave(method, initial_condition, t=0):
    u, g, conditions = initial_condition()
    cs = method(u, g, conditions)

    m_x = np.mgrid[slice(conditions["x_min"], conditions["x_max"], conditions["dx"])]
    m_y = cs[..., t]
    
    pl.plot(m_x, m_y, "b-")
    pl.grid()
    delta = 0.05
    pl.axis([m_x.min(), m_x.max(), cs.min() - delta, cs.max() + delta])
    pl.show()
    
w = widgets.interactive(show_wave,
                        method={'Explicit Flow': explicit_flow_solve, 
                                'Explicit Counter Flow': explicit_counter_flow_solve, 
                                'Leapfrog': leapfrog,
                                'Implicit Counter Flow': implicit_counter_flow},
                        initial_condition={'Flash light': get_flash_light, 
                                           'Steps': get_steps, 
                                           'Waves': get_waves, 
                                           'Rhombus': get_rhombus,
                                          'Test': get_test},
                        t=(0, 99)
                       )
display(w)

interactive(children=(Dropdown(description='method', options={'Explicit Flow': <function explicit_flow_solve at 0x10a3438c8>, 'Explicit Counter Flow': <function explicit_counter_flow_solve at 0x112384400>, 'Leapfrog': <function leapfrog at 0x11243a048>, 'Implicit Counter Flow': <function implicit_counter_flow at 0x1122eff28>}, value=<function explicit_flow_solve at 0x10a3438c8>), Dropdown(description='initial_condition', options={'Flash light': <function get_flash_light at 0x1126d3e18>, 'Steps': <function get_steps at 0x1126d38c8>, 'Waves': <function get_waves at 0x1126d3620>, 'Rhombus': <function get_rhombus at 0x1126cc6a8>, 'Test': <function get_test at 0x1126cc2f0>}, value=<function get_flash_light at 0x1126d3e18>), IntSlider(value=0, description='t', max=99), Output()), _dom_classes=('widget-interact',))